## Our clustering algorithm evaluation
Evaluating our clustering algorithm on bookswagon.com pages. The aim is to calculate precision and recall for "book details" cluster and the "catalog" cluster in bookswagon.com.

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import ast
FILEPATH = '../../../datasets/blackwells.csv'
FILEPATH

'../../../datasets/blackwells.csv'

In [2]:
df = pd.read_csv(FILEPATH, converters={'bitset': ast.literal_eval, 'tag_count': ast.literal_eval})

## Data analisys
Some preliminary analisys of the dataset

In [13]:
print("First 5 rows")
print("------------")
df.head()

First 5 rows
------------


url  \
0           https://blackwells.co.uk/bookshop/basket   
1          https://blackwells.co.uk/bookshop/search/   
2             https://blackwells.co.uk/bookshop/home   
3  https://blackwells.co.uk/bookshop/product/9781...   
4          https://blackwells.co.uk/bookshop/mapping   

                                referer_url  \
0    https://blackwells.co.uk/bookshop/home   
1    https://blackwells.co.uk/bookshop/home   
2    https://blackwells.co.uk/bookshop/home   
3    https://blackwells.co.uk/bookshop/home   
4  https://blackwells.co.uk/bookshop/basket   

                                                 src  \
0  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
1  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
2  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
3  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
4  \n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" c...   

               shingle_vector    label  \
0    (0, 1, 5, 1, 1, 6, 3, 1)      NaN   
1    (0, 1, 5, 1, 1, 0, 3, 0)     list   
2    (0, 1, 0, 1, 0, 0, 3, 1)      NaN   
3    (0, 1, 1, 1, 1, 0, 0, 1)  product   
4  (2, 22, 1, 1, 7, 15, 7, 5)      NaN   

                                           tag_count  \
0  [0.0019569471624266144, 0.0019569471624266144,...   
1  [0.0012970168612191958, 0.0012970168612191958,...   
2  [0.0011655011655011655, 0.0011655011655011655,...   
3  [0.0008116883116883117, 0.0008116883116883117,...   
4  [0.008333333333333333, 0.008333333333333333, 0...   

                                              bitset  
0  [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...  
1  [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, ...  
2  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...  
3  [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...

In [14]:
print("No. of rows and columns")
print("-----------------------")
df.shape

No. of rows and columns
-----------------------


(10919, 7)

In [15]:
print("Check null values")
print("-----------------")
df.isnull().any().any()

Check null values
-----------------


True

In [16]:
print("Check duplicate values")
print("----------------------")
len(df['url'].unique()) != df.shape[0]

Check duplicate values
----------------------


False

In [17]:
print("DataFrame column types")
print("----------------------")
df.info()

DataFrame column types
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10919 entries, 0 to 10918
Data columns (total 7 columns):
url               10919 non-null object
referer_url       10919 non-null object
src               10919 non-null object
shingle_vector    10919 non-null object
label             10899 non-null object
tag_count         10919 non-null object
bitset            10919 non-null object
dtypes: object(7)
memory usage: 597.2+ KB


In [18]:
print("Some stats")
print("----------------")
df.describe()

Some stats
----------------


url  \
count                                               10919   
unique                                              10919   
top     https://blackwells.co.uk/bookshop/product/Roma...   
freq                                                    1   

                                   referer_url  \
count                                    10919   
unique                                    6375   
top     https://blackwells.co.uk/bookshop/home   
freq                                        12   

                                                      src  \
count                                               10919   
unique                                              10525   
top     \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
freq                                                    7   

                  shingle_vector    label  \
count                      10919    10899   
unique                        73        2   
top     (0, 1, 5, 0, 1, 0, 3, 0)  product   
freq                        2197    10405   

                                                tag_count  \
count                                               10919   
unique                                               9133   
top     [0.00088261253309797, 0.00088261253309797, 0.0...   
freq                                                  946   

                                                   bitset  
count                                               10919  
unique                                               8041  
top     [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, ...  
freq                                                  951

In [19]:
fmt_string = 'There are {} row with {} label'
print(fmt_string.format(len(df[df['label'].isnull()]),'no'))
print(fmt_string.format(len(df[df['label']=='product']), 'product'))
print(fmt_string.format(len(df[df['label']=='list']), 'list'))

There are 20 row with no label
There are 10405 row with product label
There are 494 row with list label


## Run MeanShift clustering algorithm

In [10]:
#add top level folder to sys.path
import sys
sys.path.append('../../../')

In [11]:
from astarwars_clustering.clustering import clusteringevaluation
from astarwars_clustering.utils import utility
from astarwars_clustering.clustering.structural_clustering import dbscanclustering, meanshiftclustering

In [36]:
type(df['bitset'])

pandas.core.series.Series

In [37]:
clustering = meanshiftclustering(0.1, df['bitset'].tolist())

Elapsed time to calculate MeanShift clustering:00:04:58.84


In [45]:

predictedLabels = msclustering.labels_
noOfClusters = np.unique(predictedLabels)
df['predicted_label'] = predictedLabels
print('There are ' + str(noOfClusters) + 'clusters')
print()
print()
print('Cluster labels:')
noOfClusters

There are [   0    1    2 ... 8037 8038 8039]clusters


Cluster labels:


array([   0,    1,    2, ..., 8037, 8038, 8039])

In [46]:
cluster_fmt = 'cluster n. {} has {} pages'
noOfPages = 0

for index ,el in enumerate(noOfClusters):
    print(cluster_fmt.format(index ,utility.count_occurrences(predictedLabels,el)))

cluster n. 0 has 951 pages
cluster n. 1 has 163 pages
cluster n. 2 has 43 pages
cluster n. 3 has 36 pages
cluster n. 4 has 35 pages
cluster n. 5 has 27 pages
cluster n. 6 has 26 pages
cluster n. 7 has 24 pages
cluster n. 8 has 23 pages
cluster n. 9 has 19 pages
cluster n. 10 has 18 pages
cluster n. 11 has 18 pages
cluster n. 12 has 17 pages
cluster n. 13 has 16 pages
cluster n. 14 has 16 pages
cluster n. 15 has 15 pages
cluster n. 16 has 15 pages
cluster n. 17 has 14 pages
cluster n. 18 has 14 pages
cluster n. 19 has 12 pages
cluster n. 20 has 11 pages
cluster n. 21 has 11 pages
cluster n. 22 has 11 pages
cluster n. 23 has 10 pages
cluster n. 24 has 10 pages
cluster n. 25 has 10 pages
cluster n. 26 has 10 pages
cluster n. 27 has 10 pages
cluster n. 28 has 9 pages
cluster n. 29 has 9 pages
cluster n. 30 has 9 pages
cluster n. 31 has 9 pages
cluster n. 32 has 9 pages
cluster n. 33 has 8 pages
cluster n. 34 has 8 pages
cluster n. 35 has 8 pages
cluster n. 36 has 8 pages
cluster n. 37 has 

cluster n. 338 has 2 pages
cluster n. 339 has 2 pages
cluster n. 340 has 2 pages
cluster n. 341 has 2 pages
cluster n. 342 has 2 pages
cluster n. 343 has 2 pages
cluster n. 344 has 2 pages
cluster n. 345 has 2 pages
cluster n. 346 has 2 pages
cluster n. 347 has 2 pages
cluster n. 348 has 2 pages
cluster n. 349 has 2 pages
cluster n. 350 has 2 pages
cluster n. 351 has 2 pages
cluster n. 352 has 2 pages
cluster n. 353 has 2 pages
cluster n. 354 has 2 pages
cluster n. 355 has 2 pages
cluster n. 356 has 2 pages
cluster n. 357 has 2 pages
cluster n. 358 has 2 pages
cluster n. 359 has 2 pages
cluster n. 360 has 2 pages
cluster n. 361 has 2 pages
cluster n. 362 has 2 pages
cluster n. 363 has 2 pages
cluster n. 364 has 2 pages
cluster n. 365 has 2 pages
cluster n. 366 has 2 pages
cluster n. 367 has 2 pages
cluster n. 368 has 2 pages
cluster n. 369 has 2 pages
cluster n. 370 has 2 pages
cluster n. 371 has 2 pages
cluster n. 372 has 2 pages
cluster n. 373 has 2 pages
cluster n. 374 has 2 pages
c

cluster n. 671 has 2 pages
cluster n. 672 has 2 pages
cluster n. 673 has 2 pages
cluster n. 674 has 2 pages
cluster n. 675 has 2 pages
cluster n. 676 has 2 pages
cluster n. 677 has 2 pages
cluster n. 678 has 2 pages
cluster n. 679 has 2 pages
cluster n. 680 has 2 pages
cluster n. 681 has 2 pages
cluster n. 682 has 2 pages
cluster n. 683 has 2 pages
cluster n. 684 has 2 pages
cluster n. 685 has 2 pages
cluster n. 686 has 2 pages
cluster n. 687 has 2 pages
cluster n. 688 has 2 pages
cluster n. 689 has 2 pages
cluster n. 690 has 2 pages
cluster n. 691 has 2 pages
cluster n. 692 has 2 pages
cluster n. 693 has 2 pages
cluster n. 694 has 2 pages
cluster n. 695 has 2 pages
cluster n. 696 has 2 pages
cluster n. 697 has 2 pages
cluster n. 698 has 2 pages
cluster n. 699 has 2 pages
cluster n. 700 has 2 pages
cluster n. 701 has 2 pages
cluster n. 702 has 2 pages
cluster n. 703 has 2 pages
cluster n. 704 has 2 pages
cluster n. 705 has 2 pages
cluster n. 706 has 2 pages
cluster n. 707 has 2 pages
c

cluster n. 1081 has 1 pages
cluster n. 1082 has 1 pages
cluster n. 1083 has 1 pages
cluster n. 1084 has 1 pages
cluster n. 1085 has 1 pages
cluster n. 1086 has 1 pages
cluster n. 1087 has 1 pages
cluster n. 1088 has 1 pages
cluster n. 1089 has 1 pages
cluster n. 1090 has 1 pages
cluster n. 1091 has 1 pages
cluster n. 1092 has 1 pages
cluster n. 1093 has 1 pages
cluster n. 1094 has 1 pages
cluster n. 1095 has 1 pages
cluster n. 1096 has 1 pages
cluster n. 1097 has 1 pages
cluster n. 1098 has 1 pages
cluster n. 1099 has 1 pages
cluster n. 1100 has 1 pages
cluster n. 1101 has 1 pages
cluster n. 1102 has 1 pages
cluster n. 1103 has 1 pages
cluster n. 1104 has 1 pages
cluster n. 1105 has 1 pages
cluster n. 1106 has 1 pages
cluster n. 1107 has 1 pages
cluster n. 1108 has 1 pages
cluster n. 1109 has 1 pages
cluster n. 1110 has 1 pages
cluster n. 1111 has 1 pages
cluster n. 1112 has 1 pages
cluster n. 1113 has 1 pages
cluster n. 1114 has 1 pages
cluster n. 1115 has 1 pages
cluster n. 1116 has 

cluster n. 1385 has 1 pages
cluster n. 1386 has 1 pages
cluster n. 1387 has 1 pages
cluster n. 1388 has 1 pages
cluster n. 1389 has 1 pages
cluster n. 1390 has 1 pages
cluster n. 1391 has 1 pages
cluster n. 1392 has 1 pages
cluster n. 1393 has 1 pages
cluster n. 1394 has 1 pages
cluster n. 1395 has 1 pages
cluster n. 1396 has 1 pages
cluster n. 1397 has 1 pages
cluster n. 1398 has 1 pages
cluster n. 1399 has 1 pages
cluster n. 1400 has 1 pages
cluster n. 1401 has 1 pages
cluster n. 1402 has 1 pages
cluster n. 1403 has 1 pages
cluster n. 1404 has 1 pages
cluster n. 1405 has 1 pages
cluster n. 1406 has 1 pages
cluster n. 1407 has 1 pages
cluster n. 1408 has 1 pages
cluster n. 1409 has 1 pages
cluster n. 1410 has 1 pages
cluster n. 1411 has 1 pages
cluster n. 1412 has 1 pages
cluster n. 1413 has 1 pages
cluster n. 1414 has 1 pages
cluster n. 1415 has 1 pages
cluster n. 1416 has 1 pages
cluster n. 1417 has 1 pages
cluster n. 1418 has 1 pages
cluster n. 1419 has 1 pages
cluster n. 1420 has 

...

cluster n. 6265 has 1 pages
cluster n. 6266 has 1 pages
cluster n. 6267 has 1 pages
cluster n. 6268 has 1 pages
cluster n. 6269 has 1 pages
cluster n. 6270 has 1 pages
cluster n. 6271 has 1 pages
cluster n. 6272 has 1 pages
cluster n. 6273 has 1 pages
cluster n. 6274 has 1 pages
cluster n. 6275 has 1 pages
cluster n. 6276 has 1 pages
cluster n. 6277 has 1 pages
cluster n. 6278 has 1 pages
cluster n. 6279 has 1 pages
cluster n. 6280 has 1 pages
cluster n. 6281 has 1 pages
cluster n. 6282 has 1 pages
cluster n. 6283 has 1 pages
cluster n. 6284 has 1 pages
cluster n. 6285 has 1 pages
cluster n. 6286 has 1 pages
cluster n. 6287 has 1 pages
cluster n. 6288 has 1 pages
cluster n. 6289 has 1 pages
cluster n. 6290 has 1 pages
cluster n. 6291 has 1 pages
cluster n. 6292 has 1 pages
cluster n. 6293 has 1 pages
cluster n. 6294 has 1 pages
cluster n. 6295 has 1 pages
cluster n. 6296 has 1 pages
cluster n. 6297 has 1 pages
cluster n. 6298 has 1 pages
cluster n. 6299 has 1 pages
cluster n. 6300 has 

cluster n. 6602 has 1 pages
cluster n. 6603 has 1 pages
cluster n. 6604 has 1 pages
cluster n. 6605 has 1 pages
cluster n. 6606 has 1 pages
cluster n. 6607 has 1 pages
cluster n. 6608 has 1 pages
cluster n. 6609 has 1 pages
cluster n. 6610 has 1 pages
cluster n. 6611 has 1 pages
cluster n. 6612 has 1 pages
cluster n. 6613 has 1 pages
cluster n. 6614 has 1 pages
cluster n. 6615 has 1 pages
cluster n. 6616 has 1 pages
cluster n. 6617 has 1 pages
cluster n. 6618 has 1 pages
cluster n. 6619 has 1 pages
cluster n. 6620 has 1 pages
cluster n. 6621 has 1 pages
cluster n. 6622 has 1 pages
cluster n. 6623 has 1 pages
cluster n. 6624 has 1 pages
cluster n. 6625 has 1 pages
cluster n. 6626 has 1 pages
cluster n. 6627 has 1 pages
cluster n. 6628 has 1 pages
cluster n. 6629 has 1 pages
cluster n. 6630 has 1 pages
cluster n. 6631 has 1 pages
cluster n. 6632 has 1 pages
cluster n. 6633 has 1 pages
cluster n. 6634 has 1 pages
cluster n. 6635 has 1 pages
cluster n. 6636 has 1 pages
cluster n. 6637 has 

cluster n. 7005 has 1 pages
cluster n. 7006 has 1 pages
cluster n. 7007 has 1 pages
cluster n. 7008 has 1 pages
cluster n. 7009 has 1 pages
cluster n. 7010 has 1 pages
cluster n. 7011 has 1 pages
cluster n. 7012 has 1 pages
cluster n. 7013 has 1 pages
cluster n. 7014 has 1 pages
cluster n. 7015 has 1 pages
cluster n. 7016 has 1 pages
cluster n. 7017 has 1 pages
cluster n. 7018 has 1 pages
cluster n. 7019 has 1 pages
cluster n. 7020 has 1 pages
cluster n. 7021 has 1 pages
cluster n. 7022 has 1 pages
cluster n. 7023 has 1 pages
cluster n. 7024 has 1 pages
cluster n. 7025 has 1 pages
cluster n. 7026 has 1 pages
cluster n. 7027 has 1 pages
cluster n. 7028 has 1 pages
cluster n. 7029 has 1 pages
cluster n. 7030 has 1 pages
cluster n. 7031 has 1 pages
cluster n. 7032 has 1 pages
cluster n. 7033 has 1 pages
cluster n. 7034 has 1 pages
cluster n. 7035 has 1 pages
cluster n. 7036 has 1 pages
cluster n. 7037 has 1 pages
cluster n. 7038 has 1 pages
cluster n. 7039 has 1 pages
cluster n. 7040 has 

cluster n. 7492 has 1 pages
cluster n. 7493 has 1 pages
cluster n. 7494 has 1 pages
cluster n. 7495 has 1 pages
cluster n. 7496 has 1 pages
cluster n. 7497 has 1 pages
cluster n. 7498 has 1 pages
cluster n. 7499 has 1 pages
cluster n. 7500 has 1 pages
cluster n. 7501 has 1 pages
cluster n. 7502 has 1 pages
cluster n. 7503 has 1 pages
cluster n. 7504 has 1 pages
cluster n. 7505 has 1 pages
cluster n. 7506 has 1 pages
cluster n. 7507 has 1 pages
cluster n. 7508 has 1 pages
cluster n. 7509 has 1 pages
cluster n. 7510 has 1 pages
cluster n. 7511 has 1 pages
cluster n. 7512 has 1 pages
cluster n. 7513 has 1 pages
cluster n. 7514 has 1 pages
cluster n. 7515 has 1 pages
cluster n. 7516 has 1 pages
cluster n. 7517 has 1 pages
cluster n. 7518 has 1 pages
cluster n. 7519 has 1 pages
cluster n. 7520 has 1 pages
cluster n. 7521 has 1 pages
cluster n. 7522 has 1 pages
cluster n. 7523 has 1 pages
cluster n. 7524 has 1 pages
cluster n. 7525 has 1 pages
cluster n. 7526 has 1 pages
cluster n. 7527 has 

cluster n. 7829 has 1 pages
cluster n. 7830 has 1 pages
cluster n. 7831 has 1 pages
cluster n. 7832 has 1 pages
cluster n. 7833 has 1 pages
cluster n. 7834 has 1 pages
cluster n. 7835 has 1 pages
cluster n. 7836 has 1 pages
cluster n. 7837 has 1 pages
cluster n. 7838 has 1 pages
cluster n. 7839 has 1 pages
cluster n. 7840 has 1 pages
cluster n. 7841 has 1 pages
cluster n. 7842 has 1 pages
cluster n. 7843 has 1 pages
cluster n. 7844 has 1 pages
cluster n. 7845 has 1 pages
cluster n. 7846 has 1 pages
cluster n. 7847 has 1 pages
cluster n. 7848 has 1 pages
cluster n. 7849 has 1 pages
cluster n. 7850 has 1 pages
cluster n. 7851 has 1 pages
cluster n. 7852 has 1 pages
cluster n. 7853 has 1 pages
cluster n. 7854 has 1 pages
cluster n. 7855 has 1 pages
cluster n. 7856 has 1 pages
cluster n. 7857 has 1 pages
cluster n. 7858 has 1 pages
cluster n. 7859 has 1 pages
cluster n. 7860 has 1 pages
cluster n. 7861 has 1 pages
cluster n. 7862 has 1 pages
cluster n. 7863 has 1 pages
cluster n. 7864 has 

In [49]:
df[df['predicted_label'] == 0]['url'].head(10)

209     https://blackwells.co.uk/bookshop/product/The-...
288     https://blackwells.co.uk/bookshop/product/Unde...
289     https://blackwells.co.uk/bookshop/product/Art-...
337     https://blackwells.co.uk/bookshop/product/Harr...
610     https://blackwells.co.uk/bookshop/product/The-...
1957    https://blackwells.co.uk/bookshop/product/Hero...
1966    https://blackwells.co.uk/bookshop/product/Stud...
1992    https://blackwells.co.uk/bookshop/product/Chin...
1997    https://blackwells.co.uk/bookshop/product/Jim-...
2003    https://blackwells.co.uk/bookshop/product/Port...
Name: url, dtype: object

In [50]:
df[df['predicted_label'] == 1]['url'].head(10)

60     https://blackwells.co.uk/bookshop/product/Roof...
80     https://blackwells.co.uk/bookshop/product/The-...
114    https://blackwells.co.uk/bookshop/product/The-...
148    https://blackwells.co.uk/bookshop/product/A-Ma...
161    https://blackwells.co.uk/bookshop/product/Hey-...
169    https://blackwells.co.uk/bookshop/product/The-...
171    https://blackwells.co.uk/bookshop/product/Swan...
174    https://blackwells.co.uk/bookshop/product/Isad...
179    https://blackwells.co.uk/bookshop/product/Best...
195    https://blackwells.co.uk/bookshop/product/A-La...
Name: url, dtype: object

In [51]:
df[df['predicted_label'] == 2]['url'].head(10)

563     https://blackwells.co.uk/bookshop/product/A-Bo...
1757    https://blackwells.co.uk/bookshop/product/Tour...
3704    https://blackwells.co.uk/bookshop/product/The-...
3717    https://blackwells.co.uk/bookshop/product/Phot...
4354    https://blackwells.co.uk/bookshop/product/Whos...
4403    https://blackwells.co.uk/bookshop/product/Nucl...
4434    https://blackwells.co.uk/bookshop/product/Chan...
4459    https://blackwells.co.uk/bookshop/product/Hamm...
4476    https://blackwells.co.uk/bookshop/product/The-...
4478    https://blackwells.co.uk/bookshop/product/Pigg...
Name: url, dtype: object

## Evaluate recall and precision

In [52]:
p1,r1=clusteringevaluation.calculate_precision_and_recall(df,clustering,'product',0)

NameError: name 'clustering' is not defined

In [53]:
p1,r1=clusteringevaluation.calculate_precision_and_recall(df,clustering,'list',0)

NameError: name 'clustering' is not defined